# Computer Vision Approach

In [ ]:

import os                                   # Iterate over dataset directories
import numpy as np                          # Linear algebra
import pandas as pd                         # Data processing (read labels CSV)
import cv2 as cv

# Opencv for image files
import pydicom                              # Read dcm files
from sklearn.cluster import MiniBatchKMeans # Create bag of visual words
from sklearn.svm import SVC                 # Classifier
import pickle                               # Serialize and save features extracted from dataset

In [ ]:

def dcmToGray(dcm):
    image = dcm.pixel_array
    if np.amax(image) != 0:
        gray = np.uint8(image/np.amax(image)*255)
    else:
        gray = np.uint8(image)
    return gray

In [ ]:
train_path =  "../input/rsna-miccai-brain-tumor-radiogenomic-classification/train"
test_path  = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/test"

In [ ]:
subdirs = ["/FLAIR", "/T1w", "/T1wCE", "/T2w"]

In [ ]:
train_size = len(next(os.walk(train_path))[1])
test_size  = len(next(os.walk(test_path))[1])
descriptor_size = 32

# Feature detector
detector = cv.ORB_create(64)

# Size of visual vocabulary
vocab_size = 2000


# Feature detector
detector =  cv.ORB_create(64)




In [ ]:
curr_features = np.array([]).reshape(0,descriptor_size)

In [ ]:
dcm  = pydicom.dcmread("../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/FLAIR/Image-101.dcm")
gray = dcmToGray(dcm)
gray=cv.resize(gray, (512, 512))

In [ ]:
keypoints, descriptors = detector.detectAndCompute(gray,None)

In [ ]:

train_size = len(next(os.walk(train_path))[1])
test_size  = len(next(os.walk(test_path))[1])
descriptor_size = 128

# Feature detector
detector = cv.SIFT_create(400)

# Size of visual vocabulary
vocab_size = 2000





In [ ]:


# Populating array of visual features with the descriptors computed by the defined detector

# Each element of this list is an array of all the descriptor arrays
# computed by the detector for every image of each sample.
features_per_sample = []

i = 0
while(len(features_per_sample) < train_size):
    # Current directory
    curr_dir = train_path + '/{0:05d}'.format(i)
                                              
                                              
    
    i += 1
    
    # If the there is no such directory, continue to the next one
    if not os.path.exists(curr_dir):
        continue
        
    # Array of descriptor array for each image of current sample
    curr_features = np.array([]).reshape(0,descriptor_size)
        
    # Process the images from each subdirectory in the current dir
    for subdir in subdirs:
        curr_subdir = curr_dir+subdir
        for filename in os.listdir(curr_subdir):
            dcm  = pydicom.dcmread(curr_subdir+'/'+filename)
            gray = dcmToGray(dcm)
            gray=cv2.resize(gray, (512, 512))
            keypoints, descriptors = detector.detectAndCompute(gray,None)
            if descriptors is not None:
                curr_features = np.vstack([curr_features, descriptors])
                
    features_per_sample.append(curr_features)

In [ ]:
all_features = np.array([]).reshape(0,descriptor_size)
for sample_features in features_per_sample:
    all_features = np.vstack([all_features, sample_features])

In [ ]:
kmeans = MiniBatchKMeans(n_clusters = vocab_size,
                         batch_size = vocab_size//10,
                         verbose    = False, 
                         init       = 'k-means++',
                         n_init     = 3,
                         max_iter   = 1)

In [ ]:
vocab = kmeans.fit(all_features)

In [ ]:
histograms = []
for sample_features in features_per_sample:
    
    sample_hist = np.zeros(vocab_size)
    n_features  = sample_features.shape[0]
    
    visual_word_indexes = vocab.predict(sample_features)
    for index in visual_word_indexes:
        sample_hist[index] += 1/n_features
        
    histograms.append(sample_hist)

X_train = np.array(histograms)

In [ ]:
# Test set
histograms = []
test_sample_ids = []
i = 0
while(len(histograms) < test_size):
    # Current directory
    curr_dir = test_path + '/{0:05d}'.format(i)
    
    i += 1
    
    # If the there is no such directory, continue to the next one
    if not os.path.exists(curr_dir):
        continue
        
    test_sample_ids.append('{0:05d}'.format(i-1))
        
    # Array of descriptor array for each image of current sample
    curr_features = np.array([]).reshape(0,descriptor_size)
        
    # Process the images from each subdirectory in the current dir
    for subdir in subdirs:
        curr_subdir = curr_dir+subdir
        for filename in os.listdir(curr_subdir):
            dcm  = pydicom.dcmread(curr_subdir+'/'+filename)
            gray = dcmToGray(dcm)
            keypoints, descriptors = detector.detectAndCompute(gray,None)
            if descriptors is not None:
                curr_features = np.vstack([curr_features, descriptors])
                
    sample_hist = np.zeros(vocab_size)
    n_features  = curr_features.shape[0]
    
    visual_word_indexes = vocab.predict(curr_features)
    for index in visual_word_indexes:
        sample_hist[index] += 1/n_features
        
    histograms.append(sample_hist)
    
X_test = np.array(histograms)
test_sample_ids = np.array(test_sample_ids)

In [ ]:
labels = pd.read_csv("../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv")
labels = labels.iloc[:,1].values

train_labels = labels[0:int(0.9*train_size)]
valid_labels = labels[int(0.9*train_size):train_size]

In [ ]:
X_valid = X_train[int(0.9*train_size):train_size,:]
X_train = X_train[0:int(0.9*train_size),:]

In [ ]:
svc = SVC(probability=True)
svc.fit(X_train, train_labels)

In [ ]:
score = svc.score(X_valid, valid_labels)
print(score)

In [ ]:
pred = svc.predict_proba(X_test)
print(pred)

In [ ]:
pd.DataFrame({"Id": list(range(0,len(preds))), "Covtype": preds}).to_csv(fname, index=False, header=True)

In [ ]:
def write_preds(fname):
    pd.DataFrame(X_train).to_csv(fname, index=False, header=True)

write_preds("varun-predict-17.csv")

In [ ]:
import csv

with open("out.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(X_train)

In [ ]:
import csv

with open("out2.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(X_test)

In [ ]:
import pandas as pd
x=pd.read_csv('out1.csv',header=None)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import tensorflow as tf
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers.core import Dense, Activation, Dropout

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
scaler=StandardScaler()
scaler.fit(X_train)
X_train=scaler.transform(X_train)
X_test=scaler.transform(X_test)
Y_train=np_utils.to_categorical(train_labels)
X_valid=scaler.transform(X_valid)

In [ ]:
input_dim = X_train.shape[1]
nb_classes = Y_train.shape[1]

In [ ]:
model = Sequential()
model.add(Dense(50, input_dim=input_dim))
model.add(Activation('softmax'))

model.add(Dense(50, input_dim=input_dim))
model.add(Activation('softmax'))




model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
print("Training...")

model.fit(X_train, Y_train, epochs=100,batch_size=5)

In [ ]:
print("Generating test predictions...")
predict_x=model.predict(X_train) 
classes_x=np.argmax(predict_x,axis=1)

In [ ]:
print(classification_report(train_labels,classes_x))

# 3d CNN model 

In [ ]:
from keras.models import Sequential
from keras.models import Model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras import optimizers, losses, activations, models
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Concatenate
from keras import applications


In [19]:
import numpy as np 
import pandas as pd 
import os
import glob
import re
import cv2

import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

from matplotlib import pyplot as plt
import math
import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling3D,MaxPool3D,MaxPooling3D,AveragePooling3D,Dense, Flatten, Dropout, GlobalAveragePooling2D, Flatten, BatchNormalization, Conv3D
from random import shuffle
from sklearn.model_selection import train_test_split

from keras import backend as K
from textwrap import wrap

In [20]:
import os
import zipfile
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

In [16]:
df = pd.read_csv("../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv")
testSub = pd.read_csv("../input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv")
df['BraTS21ID5'] = [format(x, '05d') for x in df.BraTS21ID]
data_path = '../input/rsna-miccai-brain-tumor-radiogenomic-classification'
train_folder = os.path.join(data_path, 'train')
df['imfolder'] = ['{:05d}'.format(s) for s in df['BraTS21ID']]
df['path'] = [os.path.join(train_folder, s) for s in df['imfolder']]

#Drop these four samples sice organizers pointed out they were not valid:
df.drop(df.index[df['BraTS21ID5'] == "00109"], inplace = True)
df.drop(df.index[df['BraTS21ID5'] == "00123"], inplace = True)
df.drop(df.index[df['BraTS21ID5'] == "00709"], inplace = True)


#Split into train and test set:
train_df, test_df = train_test_split(df, test_size=0.20)
train_df.iloc[0:5]

In [163]:
def loadimage(path, img_size=128):
    dicom_image=dcmread(path)
#     dicom = pydicom.read_file(path)
#     data = dicom.pixel_array
#     data = cv2.resize(data, (img_size, img_size))
    pixel_array = dicom_image.pixel_array[80:400, 100:420]
    if np.max(pixel_array) == 0:#Blank image
        return []
    else:    
        pixel_array = cv2.resize(pixel_array, (img_size, img_size))
        pixel_array = np.array(pixel_array, dtype = float)
        pixel_array /= 4095
    return pixel_array

In [164]:
def loadimages(scan_id, mtype="FLAIR", split="train"):

    files = sorted(glob.glob(f"{data_path}/{split}/{scan_id}/{mtype}/*.dcm"), 
       key=lambda var:[int(x) if x.isdigit() else x for x in re.findall(r'[^0-9]|[0-9]+', var)])
    images=[]
    
#     if scan_id in pick_idx: 
#         #print('happens')
#         img_out = np.stack([load_dicom_image(f, rotate_img=rotate_img) for f in [files[kk] for kk in pick_idx[scan_id]]]).T    
#     else:

    for f in files:
            if len(images)==0:
                images = np.expand_dims(loadimage(f),-1)
            else:
                if(len(loadimage(f))==0):
                    pass
                else:
                    images = np.append(images,np.expand_dims(loadimage(f),-1),axis=-1)
    images=np.nan_to_num(images)
    try:
        images=images[:,:,np.linspace(0,24,24,dtype=int)]
        output = np.array(images,dtype=float)
        return(output)
        
    except:
        return []
        
    
        
    
    
    

In [165]:
X_train=[]
Y_train=[]
z=train_df["MGMT_value"].values
u=train_df['BraTS21ID5'].values
for i in range(len(u)):
    x=loadimages(u[i], mtype="FLAIR", split="train")
    if(len(x)!=0):
        X_train.append(x)
        Y_train.append(z[i])
        print(i)

In [166]:
X_valid=[]
Y_valid=[]
z=test_df["MGMT_value"].values
u=test_df['BraTS21ID5'].values
for i in range(len(u)):
    x=loadimages(u[i], mtype="FLAIR", split="train")
    if(len(x)!=0):
        X_valid.append(x)
        Y_valid.append(z[i])
        print(i)

In [175]:
X_train=np.array(X_train)
Y_train=np.array(Y_train)
X_valid=np.array(X_valid)
Y_valid=np.array(Y_valid)

In [161]:
# X_train=[]
# Y_train=[]
# z=train_df["MGMT_value"].values
# u=train_df['BraTS21ID5'].values
# for i in range(len(u)):
#     x=loadimages(u[i], mtype="T1wCE", split="train")
#     if(len(x)!=0):
#         X_train.append(x)
#         Y_train.append(z[i])
#         print(i)

In [162]:
# X_valid=[]
# Y_valid=[]
# z=test_df["MGMT_value"].values
# u=test_df['BraTS21ID5'].values
# for i in range(len(u)):
#     x=loadimages(u[i], mtype="T1wCE", split="train")
#     if(len(x)!=0):
#         X_valid.append(x)
#         Y_valid.append(z[i])
#         print(i)

In [108]:
# X_train=np.array(X_train)
# Y_train=np.array(Y_train)
# X_valid=np.array(X_valid)
# Y_valid=np.array(Y_valid)

In [172]:
def get_model(width=128, height=128, depth=24):
    """Build a 3D convolutional neural network model."""

    inputs = keras.Input((width, height, depth, 1))

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.1)(x)   
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(x)
    x = layers.GlobalAveragePooling3D()(x)
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(units=512, activation="relu")(x)

    outputs = layers.Dense(units=1, activation="sigmoid")(x)

    # Define the model.
    model = keras.Model(inputs, outputs, name="3dcnn")
    return model

In [177]:
model = get_model(width=128, height=128, depth=24)
model.summary()

In [178]:
import gc
gc.collect()
model.compile(
    loss="binary_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    metrics=[AUC(name='auc'),"acc"],
)
epochs = 100
gc.collect()
model.fit(
    X_train,
    Y_train,
    batch_size=10,
    validation_data=(X_valid, Y_valid),
    epochs=epochs,
    shuffle=True,
)

# Model prediction

In [ ]:
# Load best weights.
#model.load_weights("3d_image_classification.h5")
prediction = model.predict(np.expand_dims(X_valid, axis=0))[0]
scores = [1 - prediction[0], prediction[0]]

class_names = ["tumor is absent", "tumor is present"]
for score, name in zip(scores, class_names):
    print(
        "This model is %.2f percent confident that  %s"
        % ((100 * score), name)
    )

# Compute the roc curve 

In [181]:
from sklearn.metrics import roc_curve,roc_auc_score

fpr , tpr , thresholds = roc_curve ( Y_valid , model.predict(X_valid))

# Compute the f1 score of the model

In [186]:
from sklearn.metrics import f1_score
l=[]
for i in pred:
    if(i[0]>0.5):
        l.append(1)
    else:
        l.append(0)

In [187]:
f1_score(Y_valid, l, average='macro')